## Example: using Linear Programming to solve for 3 latent factors `w`

Please note that the last part of this example, "Permutation invariance in linear programming vs. MultiCCA in R and `pmd`" requires that `inpt3` be exactly as printed.

In [ ]:
import pandas as pd
import numpy as np

import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append("../sparsecca/")
from sparsecca import lp_pmd
 

In [ ]:
# example inputs
inpt1 = pd.read_csv("tests/data/multicca1.csv", sep=",", index_col=0).values
inpt2 = pd.read_csv("tests/data/multicca2.csv", sep=",", index_col=0).values[:, -5:]
inpt3 = np.random.normal(size=inpt2.shape)

penalties = [1.5, 1.5, 1.5]

In [ ]:
inpt3

In [ ]:
weights, svd_init = lp_pmd(
    datasets=[inpt1, inpt2], # match feature dimension for now
    penalties=penalties[:2],
    K=3,
    standardize=True,
    mimic_R=True
)

In [ ]:
weights

## Correlation between pmd results and lp results

In [ ]:
from scipy.stats import spearmanr
from sparsecca import multicca_pmd

### high correlation between pmd and lp when `n=2`

Let `n` equal the number of datasets.

In [ ]:
datasets = [inpt1, inpt2]
ws_lp, _ = lp_pmd(datasets, penalties[:2])
ws_r, _ = multicca_pmd(datasets, penalties[:2])

In [ ]:
print(spearmanr(ws_lp[0], ws_r[0]))
print(spearmanr(ws_lp[1], ws_r[1]))

Note that this is the same as taking the first latent factor when K=3, as per the implementation in Witten 2009.

In [ ]:
print(spearmanr(weights[0].T[0], ws_r[0]))
print(spearmanr(weights[1].T[0], ws_r[1]))

### arbitrarily lower correlation when `n=3`

Compare when `n=3`. Note that this is subject to the same warning as in the bottom of the notebook.

In [ ]:
datasets = [inpt1, inpt2, inpt3]
ws_lp, _ = lp_pmd(datasets, penalties[:3])
ws_r, _ = multicca_pmd(datasets, penalties[:3])

In [ ]:
print(spearmanr(ws_lp[0], ws_r[0]))
print(spearmanr(ws_lp[1], ws_r[1]))
print(spearmanr(ws_lp[2], ws_r[2]))

#### Optimization of the objective function

Size of the objective function, which was maximized:

In [ ]:
print('1st pair:', ws_lp[0].T @ inpt1.T @ inpt2 @ ws_lp[1])
print('2nd pair:', ws_lp[1].T @ inpt2.T @ inpt3 @ ws_lp[2])
print('3rd pair:', ws_lp[2].T @ inpt3.T @ inpt1 @ ws_lp[0])
print('sum', ws_lp[0].T @ inpt1.T @ inpt2 @ ws_lp[1] + ws_lp[1].T @ inpt2.T @ inpt3 @ ws_lp[2] + ws_lp[2].T @ inpt3.T @ inpt1 @ ws_lp[0])

In [ ]:
print('1st pair:', ws_r[0].T @ inpt1.T @ inpt2 @ ws_r[1])
print('2nd pair:', ws_r[1].T @ inpt2.T @ inpt3 @ ws_r[2])
print('3rd pair:', ws_r[2].T @ inpt3.T @ inpt1 @ ws_r[0])
print('sum', ws_r[0].T @ inpt1.T @ inpt2 @ ws_r[1] + ws_r[1].T @ inpt2.T @ inpt3 @ ws_r[2] + ws_r[2].T @ inpt3.T @ inpt1 @ ws_r[0])

L2 norm of the latent factors, constrained.

In [ ]:
np.linalg.norm(ws_lp)

In [ ]:
np.linalg.norm(ws_r)

Neither linear programming nor manual convergence consistently produces a better optimization of the objective function in this small test case.

## Permutation invariance in linear programming vs. MultiCCA in R and `pmd`

In [ ]:
def test_weights(weights_a, weights_b, perm_b: list[int], dec=5):
    """Tests whether `weights_a` and `weights_b` are the same given the permutation order of b.

    Parameters:
        weights_a: output of lp_pmd 
        weights_b: output of lp_pmd permuted Xn
                   -> weights are of type np.ndarray in shape (N, f, K)
                    - N: len(Xn) datasets
                    - f: amount of features
                    - K: amount of MCPs
        perm_b:    order of the datasets used to generate a, in b
        dec:       decimals to which weights should be rounded to account for numerical tolerance

    Returns:
        boolean: True if rounded weights are the same, else False
    """
    weights_a_rounded = [list(w_a.round(decimals=dec).flatten()) for w_a in weights_a]
    weights_b_rounded = [list(w_b.round(decimals=dec).flatten()) for w_b in weights_b]
    
    weights_b_ordered = []
    for o in perm_b:
        weights_b_ordered.append(weights_b_rounded[o])

    return (weights_a_rounded==weights_b_ordered)

In [ ]:
datasets = [inpt1, inpt2, inpt3]
# original dataset with perm 1, 2, 0
datasets_perm = [inpt3, inpt1, inpt2]

### linear programming

In [ ]:
ws_lp, _ = lp_pmd(datasets, penalties)
ws_lp_perm, _ = lp_pmd(datasets_perm, penalties)

In [ ]:
ws_lp

In [ ]:
ws_lp_perm

In [ ]:
test_weights(ws_lp, ws_lp_perm, [1,2,0], dec=5)

As we can see, the weights are the same with merely the order permuted as is appropriate.

### R (in python)

In [ ]:
ws_r, _ = multicca_pmd(datasets, penalties)
ws_r_perm, _ = multicca_pmd(datasets_perm, penalties)

In [ ]:
ws_r

In [ ]:
ws_r_perm

Test with decimal tolerance 5.

In [ ]:
test_weights(np.array(ws_r), np.array(ws_r_perm), [1,2,0])

Test with decimal tolerance 1.

In [ ]:
test_weights(np.array(ws_r), np.array(ws_r_perm), [1,2,0], dec=1)

Test the equivalent negative solution.

In [ ]:
test_weights(np.array(ws_r), -np.array(ws_r_perm), [1,2,0], dec=1)

**Note**: linear programming will always solve for the same objective function regardless of the order of the inputs, but the custom PMD implementation in `multicca_pmd` will not. Depending on the dataset, the solution may converge similarly regardless of order (with only a difference in sign), or the solution may converge to a completely different local minima given a different order.

## UC data

In [6]:
import anndata as ad
from sparsecca import multicca_pmd, lp_pmd
import numpy as np

In [7]:
adata_in = ad.read_h5ad("../tmp_dialogue_experiments/data/uc-dialogue-reduced.h5ad")

In [11]:
adata_in.X = np.nan_to_num(adata_in.X, 0)
adata = adata_in

shared_samples = (
    set(adata.obs.query("cell_type == 'immune'")["Sample"])
    .intersection(set(adata.obs.query("cell_type == 'fibroblast'")["Sample"]))
    .intersection(set(adata.obs.query("cell_type == 'epithelial'")["Sample"]))
)

adata = adata[adata.obs["Sample"].isin(shared_samples)]

adata_in = adata

data_epi = adata_in[adata_in.obs["cell_type"]=="epithelial"].X
data_imm = adata_in[adata_in.obs["cell_type"]=="immune"].X
data_fib =  adata_in[adata_in.obs["cell_type"]=="fibroblast"].X



In [12]:
uc_datasets = [data_epi, data_imm, data_fib]

## do PMD on UC data

### do PCA


In [14]:
from pertpy import tools

In [ ]:
dl = tools.Dialogue()
dl.load(adata_in, )

In [13]:
penalties = [1.5, 1.5, 1.5]
# ws_lp, _ = lp_pmd(uc_datasets, penalties)
ws_r, _ = multicca_pmd(uc_datasets, penalties)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3541 is different from 2112)